CREATE WIDGET DROPDOWN ProductionDepartment DEFAULT 'All' CHOICES SELECT productiondepartmentname from reference.productionDepartment
CREATE WIDGET DROPDOWN ResponsibleDepartment DEFAULT 'All' CHOICES SELECT responsibledepartmentname from reference.responsibleDepartment
CREATE WIDGET DROPDOWN Division DEFAULT 'All' CHOICES SELECT division from reference.division
CREATE WIDGET DROPDOWN BusinessUnit DEFAULT 'All' CHOICES SELECT businessunit from reference.businessUnit
CREATE WIDGET DROPDOWN Workflow DEFAULT 'All' CHOICES SELECT workflowname from reference.workflow
CREATE WIDGET DROPDOWN Step DEFAULT 'All' CHOICES SELECT stepname from reference.workflowsteps where workflowname in(:Workflow)
CREATE WIDGET DROPDOWN AgencyState DEFAULT 'All' CHOICES SELECT state from reference.agencyState
CREATE WIDGET DROPDOWN AgencyRegion DEFAULT 'All' CHOICES SELECT regionname from reference.agencyRegion
CREATE WIDGET DROPDOWN Copmany DEFAULT 'All' CHOICES SELECT companyname from reference.company
CREATE WIDGET DROPDOWN CompletedStatus DEFAULT 'All' CHOICES SELECT completedstatus from reference.completedstatus

In [0]:
%sql
with goldFiltered as (select
       coalesce(productiondepartmentname, 'ERROR') productionDepartmentName,
       coalesce(responsibledepartmentname,'ERROR') responsibleDepartmentName,
       coalesce(businessunitname, 'ERROR')   BusinessUnitName,
       coalesce(divisionname, 'ERROR')       DivisionName,
       coalesce(stepname, 'ERROR')           stepName,
       coalesce(workflowname, 'ERROR')       workflowName,
       coalesce(companyname, 'ERROR')       companyName,
      coalesce(agencyState, 'ERROR')       agencyState,
      coalesce(RegionName, 'ERROR')       agencyRegion,
       excludedfrommaster,
       completed,
       dueDateDate
from dev.gold_dbmars.workflow
WHERE  (divisionName in(:Division) or :Division='All')
and (businessUnitName in(:BusinessUnit) or :BusinessUnit='All')
and (productiondepartmentname in(:ProductionDepartment) or :ProductionDepartment='All')
and (responsibledepartmentname in(:ResponsibleDepartment) or :ResponsibleDepartment='All')
and (workflowName in(:Workflow) or :Workflow='All')
and (stepName in(:Step) or :Step='All')
and (companyname in(:Company) or :Company='All')
and (agencystate in(:AgencyState) or :AgencyState='All') 
and (regionName in(:AgencyRegion) or :AgencyRegion='All')
and (completedStatus in(:CompletedStatus) or :CompletedStatus='All')
and (completedByusername in(:CompletedBy) or :CompletedBy='All')
and (assignedTousername in(:AssignedTo) or :AssignedTo='All') ),


--PRIOR DUE

priorDue as (
  select
    productionDepartmentName,
    responsibleDepartmentName,
    BusinessUnitName,
    DivisionName,
    stepName,
    workflowName,
    companyName,
    agencyState,
    agencyRegion,
    Sum(CASE WHEN excludedfrommaster = 'true' THEN 1 ELSE 0 END) as internalPriorDue,
    Sum(CASE WHEN excludedfrommaster = 'false' THEN 1 ELSE 0 END) as externalPriorDue
from goldFiltered
WHERE (dueDateDate < case when :StartDate_YYYYMMDD != 'Now' then cast(:StartDate_YYYYMMDD as Date) else current_date() end)
GROUP  BY productionDepartmentName,
    responsibleDepartmentName,
    BusinessUnitName,
    DivisionName,
    stepName,
    workflowName,
    companyName,
    agencyState,
    agencyRegion
),

--CURRENT DUE

currentDue as (
  select
    productionDepartmentName,
    responsibleDepartmentName,
    BusinessUnitName,
    DivisionName,
    stepName,
    workflowName,
    companyName,
    agencyState,
    agencyRegion,
       Sum(CASE WHEN excludedfrommaster = 'true' THEN 1 ELSE 0 END) as internalDueThisWeek,
       Sum(CASE WHEN excludedfrommaster = 'false' THEN 1 ELSE 0 END) as externalDueThisWeek
from goldFiltered
WHERE (dueDateDate between 
      case when :StartDate_YYYYMMDD != 'Now' then cast(:StartDate_YYYYMMDD as Date) else current_date() end 
    and 
     case when :EndDate_YYYYMMDD != 'Now' then cast(:EndDate_YYYYMMDD as Date) else current_date() end) 
GROUP  BY 
    productionDepartmentName,
    responsibleDepartmentName,
    BusinessUnitName,
    DivisionName,
    stepName,
    workflowName,
    companyName,
    agencyState,
    agencyRegion),

--PRIOR COMPLETED

priorCompleted as(
select
    productionDepartmentName,
    responsibleDepartmentName,
    BusinessUnitName,
    DivisionName,
    stepName,
    workflowName,
    companyName,
    agencyState,
    agencyRegion,
    Sum(CASE WHEN excludedfrommaster = 'true' THEN 1 ELSE 0 END) as internalPriorCompleted,
    Sum(CASE WHEN excludedfrommaster = 'false' THEN 1 ELSE 0 END) as externalPriorCompleted
from goldFiltered
WHERE   dueDateDate < case when :StartDate_YYYYMMDD != 'Now' then cast(:StartDate_YYYYMMDD as Date) else current_date() end
and (completed between 
      case when :StartDate_YYYYMMDD != 'Now' then cast(:StartDate_YYYYMMDD as Date) else current_date() end 
    and 
     case when :EndDate_YYYYMMDD != 'Now' then cast(:EndDate_YYYYMMDD as Date) else current_date() end)
GROUP  BY productionDepartmentName,
    responsibleDepartmentName,
    BusinessUnitName,
    DivisionName,
    stepName,
    workflowName,
    companyName,
    agencyState,
    agencyRegion),

--CURRENT COMPLETED

currentCompleted as (
  select
    productionDepartmentName,
    responsibleDepartmentName,
    BusinessUnitName,
    DivisionName,
    stepName,
    workflowName,
    companyName,
    agencyState,
    agencyRegion,
    Sum(CASE WHEN excludedfrommaster = 'true' THEN 1 ELSE 0 END) as internalCompletedThisWeek,
    Sum(CASE WHEN excludedfrommaster = 'false' THEN 1 ELSE 0 END) as externalCompletedThisWeek
from goldFiltered
WHERE  (dueDateDate between 
      case when :StartDate_YYYYMMDD != 'Now' then cast(:StartDate_YYYYMMDD as Date) else current_date() end 
    and 
     case when :EndDate_YYYYMMDD != 'Now' then cast(:EndDate_YYYYMMDD as Date) else current_date() end)
and (completed between 
      case when :StartDate_YYYYMMDD != 'Now' then cast(:StartDate_YYYYMMDD as Date) else current_date() end 
    and 
     case when :EndDate_YYYYMMDD != 'Now' then cast(:EndDate_YYYYMMDD as Date) else current_date() end)
GROUP  BY   
    productionDepartmentName,
    responsibleDepartmentName,
    BusinessUnitName,
    DivisionName,
    stepName,
    workflowName,
    companyName,
    agencyState,
    agencyRegion),

--FUTURE COMPLETED

futureCompleted as (
  select
    productionDepartmentName,
    responsibleDepartmentName,
    BusinessUnitName,
    DivisionName,
    stepName,
    workflowName,
    companyName,
    agencyState,
    agencyRegion,
    Sum(CASE WHEN excludedfrommaster = 'true' THEN 1 ELSE 0 END) as internalFutureCompleted,
    Sum(CASE WHEN excludedfrommaster = 'false' THEN 1 ELSE 0 END) as externalFutureCompleted
from goldFiltered
WHERE (dueDateDate > case when :EndDate_YYYYMMDD != 'Now' then cast(:EndDate_YYYYMMDD as Date) else current_date() end) 
and (completed > case when :EndDate_YYYYMMDD != 'Now' then cast(:EndDate_YYYYMMDD as Date) else current_date() end) 
GROUP  BY   
    productionDepartmentName,
    responsibleDepartmentName,
    BusinessUnitName,
    DivisionName,
    stepName,
    workflowName,
    companyName,
    agencyState,
    agencyRegion)


select pr.productionDepartmentName,
    pr.responsibleDepartmentName,
    pr.BusinessUnitName,
    pr.DivisionName,
    pr.stepName,
    pr.workflowName,
    pr.companyName,
    pr.agencyState,
    pr.agencyRegion,
    pr.internalPriorDue,
    pr.externalPriorDue,
    cd.internalDueThisWeek,
    cd.externalDueThisWeek,
    pc.internalPriorCompleted,
    pc.externalPriorCompleted,
    cc.internalCompletedThisWeek,
    cc.externalCompletedThisWeek,
    fc.internalFutureCompleted,
    fc.externalFutureCompleted
from priorDue pr 
full outer join currentDue cd on 
    concat(pr.productionDepartmentName,pr.responsibleDepartmentName,pr.BusinessUnitName, pr.DivisionName,pr.stepName,pr.workflowName,pr.companyName,pr.agencyState,pr.agencyRegion)=concat(cd.productionDepartmentName,cd.responsibleDepartmentName,cd.BusinessUnitName,cd.DivisionName,cd.stepName,cd.workflowName,cd.companyName,cd.agencyState,cd.agencyRegion)
full outer join priorCompleted pc on
    concat(pr.productionDepartmentName,pr.responsibleDepartmentName,pr.BusinessUnitName, pr.DivisionName,pr.stepName,pr.workflowName,pr.companyName,pr.agencyState,pr.agencyRegion)=concat(pc.productionDepartmentName,pc.responsibleDepartmentName,pc.BusinessUnitName,pc.DivisionName,pc.stepName,pc.workflowName,pc.companyName,pc.agencyState,pc.agencyRegion)
full outer join currentCompleted cc on
    concat(pr.productionDepartmentName,pr.responsibleDepartmentName,pr.BusinessUnitName, pr.DivisionName,pr.stepName,pr.workflowName,pr.companyName,pr.agencyState,pr.agencyRegion)=concat(cc.productionDepartmentName,cc.responsibleDepartmentName,cc.BusinessUnitName,cc.DivisionName,cc.stepName,cc.workflowName,cc.companyName,cc.agencyState,cc.agencyRegion)
full outer join futureCompleted fc on
    concat(pr.productionDepartmentName,pr.responsibleDepartmentName,pr.BusinessUnitName, pr.DivisionName,pr.stepName,pr.workflowName,pr.companyName,pr.agencyState,pr.agencyRegion)=concat(fc.productionDepartmentName,fc.responsibleDepartmentName,fc.BusinessUnitName,fc.DivisionName,fc.stepName,fc.workflowName,fc.companyName,fc.agencyState,fc.agencyRegion)


In [0]:
pandas_df=_sqldf.toPandas()
df=pandas_df.to_json()
dbutils.notebook.exit(df)